## Find restaurants near a business hub for site suitability in Chicago

In [35]:
from pyplaces import overture_maps as om
import folium

In [3]:
def plot_geodataframes(gdfs, zoom_start=15, popup_columns=None):
    if not isinstance(gdfs, list):
        gdfs = [gdfs]
    if popup_columns is None:
        popup_columns = [None] * len(gdfs)

    # Filter out empty GeoDataFrames
    gdfs_filtered = [(gdf, popup_columns[i]) for i, gdf in enumerate(gdfs) if not gdf.empty]

    if not gdfs_filtered:
        print("All GeoDataFrames are empty.")
        return None

    # Center the map at the centroid of the first non-empty GeoDataFrame
    center = gdfs_filtered[0][0].geometry.unary_union.centroid
    m = folium.Map(location=[center.y, center.x], zoom_start=zoom_start, tiles="CartoDB positron")

    for gdf, popup_col in gdfs_filtered:
        # Check if all geometries are Points
        if gdf.geometry.geom_type.isin(['Point']).all():
            for _, row in gdf.iterrows():
                coords = [row.geometry.y, row.geometry.x]
                popup = str(row[popup_col]) if popup_col and popup_col in row else None
                folium.CircleMarker(
                    location=coords,
                    radius=1,   # size of the dot
                    color='green',
                    fill=True,
                    fill_color='green',
                    fill_opacity=1,
                    popup=popup
                ).add_to(m)
        else:
            # Otherwise treat as polygon or line geometries
            if popup_col and popup_col in gdf.columns:
                folium.GeoJson(
                    gdf,
                    tooltip=folium.GeoJsonTooltip(fields=[popup_col], aliases=['Name:']),
                    popup=folium.GeoJsonPopup(fields=[popup_col])
                ).add_to(m)
            else:
                folium.GeoJson(gdf).add_to(m)

    return m

In [38]:
# Get all restaurants in the Near West Side
gdf_food = om.overture_places_from_place("Near West Side, Chicago, IL",filters=[("categories.primary","contains","eat_and_drink")])

# Filter buildings by height
gdf_buildings = om.overture_buildings_from_place(
    address="Near West Side, Chicago, IL",
    filters=[("num_floors", ">=", 5)]
)

In [41]:
gdf_food.rename(columns={"geom":"geometry"},inplace=True)
gdf_food.set_geometry("geometry",inplace=True)

In [40]:
gdf_food[["fsq_place_id","name","geometry"]]

,fsq_place_id,name,latitude,longitude,address,locality,region,postcode,admin_region,post_town,...,website,email,facebook_id,instagram,twitter,fsq_category_ids,fsq_category_labels,placemaker_url,geom,bbox
0,50331877e4b0b4389f01c15e,ShonPierre Cooks At Impure's,41.863576,-87.636382,None,Chicago,IL,60616,None,None,...,None,None,NaN,None,None,[4d4b7105d754a06374d81259],[Dining and Drinking > Restaurant],https://foursquare.com/placemakers/review-plac...,POINT (-87.63638 41.86358),"{'xmin': -87.63638204078718, 'ymin': 41.863576..."
53,63ab2f89b37abe707400498d,Tempesta Market,41.876138,-87.638435,433 W Van Buren St,Chicago,IL,60607,None,None,...,None,None,NaN,None,None,[4d4b7105d754a06374d81259],[Dining and Drinking > Restaurant],https://foursquare.com/placemakers/review-plac...,POINT (-87.63844 41.87614),"{'xmin': -87.638435, 'ymin': 41.876138, 'xmax'..."
54,59723045b9ac383dd36cc305,Little Toasted,41.877512,-87.639077,300 S Riverside Plz,Chicago,IL,60606,None,None,...,https://littletoasted.com,None,NaN,None,None,[4d4b7105d754a06374d81259],[Dining and Drinking > Restaurant],https://foursquare.com/placemakers/review-plac...,POINT (-87.63908 41.87751),"{'xmin': -87.63907723878081, 'ymin': 41.877511..."
55,5dd981775f2e478e6da9d122,Snuggery,41.878762,-87.638924,222 S Riverside Plz Ste LL5,Chicago,IL,60606,None,None,...,http://www.snuggerychicago.com,None,NaN,None,None,"[4bf58dd8d48988d116941735, 4d4b7105d754a06374d...","[Dining and Drinking > Bar, Dining and Drinkin...",https://foursquare.com/placemakers/review-plac...,POINT (-87.63892 41.87876),"{'xmin': -87.63892372304649, 'ymin': 41.878762..."
56,51cb430c498e9fd013257f55,Protein Bar & Kitchen,41.880170,-87.638850,120 S Riverside Plz,Chicago,IL,60606,None,None,...,http://theproteinbar.com,info@theproteinbar.com,6.455323e+14,None,theproteinbar,[4d4b7105d754a06374d81259],[Dining and Drinking > Restaurant],https://foursquare.com/placemakers/review-plac...,POINT (-87.63885 41.88017),"{'xmin': -87.63885, 'ymin': 41.88017, 'xmax': ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
293,4f3202b619833175d6097065,Super Star Thai/Chinese Cuisine,41.867680,-87.685825,1123 S Western Ave,Chicago,IL,60612,None,None,...,None,None,NaN,None,None,[4d4b7105d754a06374d81259],[Dining and Drinking > Restaurant],https://foursquare.com/placemakers/review-plac...,POINT (-87.68583 41.86768),"{'xmin': -87.68582518219047, 'ymin': 41.867680..."
294,4f3204b719833175d60a354a,Pine Grill,41.866401,-87.685800,1201 S Western Ave,Chicago,IL,60608,None,None,...,None,None,NaN,None,None,[4d4b7105d754a06374d81259],[Dining and Drinking > Restaurant],https://foursquare.com/placemakers/review-plac...,POINT (-87.6858 41.8664),"{'xmin': -87.68579955265548, 'ymin': 41.866401..."
295,15feb741b0f44c9132dd7e63,Tommy's Hot Dogs,41.865288,-87.685785,2390 W Ogden Ave,Chicago,IL,60608,None,None,...,http://www.tommysredhots.com,None,NaN,None,None,[4d4b7105d754a06374d81259],[Dining and Drinking > Restaurant],https://foursquare.com/placemakers/review-plac...,POINT (-87.68578 41.86529),"{'xmin': -87.68578475391065, 'ymin': 41.865287..."
296,4c9e532c2fb1a143af6ee940,Amazing Edibles Catering,41.862789,-87.686838,2419 W 14th St,Chicago,IL,60608,None,None,...,http://www.cateramazing.com,sales@amazingediblescatering.com,NaN,None,amazingedibles,"[4d4b7105d754a06374d81259, 4bf58dd8d48988d1289...","[Dining and Drinking > Restaurant, Dining and ...",https://foursquare.com/placemakers/review-plac...,POINT (-87.68684 41.86279),"{'xmin': -87.68683843521615, 'ymin': 41.862788..."


In [39]:
gdf_buildings[["id","subtype","geometry"]]

,id,geometry,bbox,version,sources,level,subtype,class,height,names,...,min_height,min_floor,facade_color,facade_material,roof_material,roof_shape,roof_direction,roof_orientation,roof_color,roof_height
1,08b2664caba0cfff0200e24d7de9b351,"POLYGON ((-87.6822 41.86477, -87.68219 41.8644...","{'xmin': -87.68220520019531, 'xmax': -87.68199...",0,"[{'property': '', 'dataset': 'OpenStreetMap', ...",NaN,None,None,21.100000,None,...,NaN,NaN,None,None,None,None,NaN,None,None,NaN
2,08b2664cabb51fff020024e93d868f0d,"POLYGON ((-87.68049 41.86572, -87.6805 41.8653...","{'xmin': -87.68051147460938, 'xmax': -87.67898...",0,"[{'property': '', 'dataset': 'OpenStreetMap', ...",NaN,commercial,office,52.800000,{'primary': 'Federal Bureau of Investigation C...,...,NaN,NaN,None,None,None,None,NaN,None,None,NaN
3,08b2664cabb03fff0200e6a859c4e513,"POLYGON ((-87.68272 41.86696, -87.6827 41.8669...","{'xmin': -87.68276977539062, 'xmax': -87.68044...",0,"[{'property': '', 'dataset': 'OpenStreetMap', ...",NaN,civic,public,49.900000,"{'primary': 'Cook County Juvenile Court', 'com...",...,NaN,NaN,None,None,None,None,NaN,None,None,NaN
4,08b2664ca945afff0200eb6ae5bb318a,"POLYGON ((-87.67532 41.86818, -87.67532 41.868...","{'xmin': -87.67532348632812, 'xmax': -87.67430...",0,"[{'property': '', 'dataset': 'OpenStreetMap', ...",NaN,education,university,33.500000,{'primary': 'Applied Health Sciences Building ...,...,NaN,NaN,None,None,None,None,NaN,None,None,NaN
5,08b2664ca9710fff0200de34c402bf72,"POLYGON ((-87.67152 41.86755, -87.6718 41.8675...","{'xmin': -87.67180633544922, 'xmax': -87.67149...",0,"[{'property': '', 'dataset': 'OpenStreetMap', ...",NaN,None,None,10.467542,{'primary': 'The Foglia Residences at Chicago ...,...,NaN,NaN,None,None,None,None,NaN,None,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
361,08b2664cada40fff02009518762742a3,"POLYGON ((-87.64176 41.88769, -87.64152 41.887...","{'xmin': -87.64208221435547, 'xmax': -87.64151...",0,"[{'property': '', 'dataset': 'OpenStreetMap', ...",NaN,residential,apartments,23.600000,None,...,NaN,NaN,None,None,None,None,NaN,None,None,NaN
362,08b2664c1ad33fff020014c1d2b87130,"POLYGON ((-87.6396 41.88595, -87.63953 41.8858...","{'xmin': -87.63971710205078, 'xmax': -87.63917...",0,"[{'property': '', 'dataset': 'OpenStreetMap', ...",-1.0,commercial,office,222.000000,"{'primary': 'River Point', 'common': None, 'ru...",...,NaN,NaN,None,None,None,None,NaN,None,None,NaN
363,08b2664cada6bfff02005a09606d1a2c,"POLYGON ((-87.64021 41.88736, -87.64059 41.887...","{'xmin': -87.64059448242188, 'xmax': -87.64005...",0,"[{'property': '', 'dataset': 'OpenStreetMap', ...",NaN,residential,apartments,111.000000,"{'primary': 'Left Bank at K Station', 'common'...",...,NaN,NaN,None,None,None,None,NaN,None,None,NaN
364,08b2664cada6dfff0200f75f8de2767f,"POLYGON ((-87.63964 41.88782, -87.63975 41.887...","{'xmin': -87.63976287841797, 'xmax': -87.63933...",0,"[{'property': '', 'dataset': 'OpenStreetMap', ...",NaN,residential,apartments,138.000000,"{'primary': 'Riverbend', 'common': None, 'rule...",...,NaN,NaN,None,None,None,None,NaN,None,None,NaN


In [44]:
plot_geodataframes([gdf_food[["fsq_place_id","name","geometry"]],gdf_buildings[["id","subtype","geometry"]],],popup_columns=["name","subtype"])

C:\Users\tedba\AppData\Local\Temp\ipykernel_9700\27719878.py:15: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  center = gdfs_filtered[0][0].geometry.unary_union.centroid
